# MULTI PRECISION A.I:


# Imports:


In [ ]:
import pandas as pd
from pandas import Timestamp
import numpy as np
import datetime


#to plot within notebook
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

%matplotlib inline

#setting figure size
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20,10

#for normalizing data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

import google_sheets_api as sheet #connection to server


In [ ]:
#AI Librarys 
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

# Download Stock Data:

In [ ]:
def getAllSheets():
    sheet_aapl = sheet.AAPL_data.get_all_values() 
    sheet_amzn = sheet.AMZN_data.get_all_values()
    sheet_csco = sheet.CSCO_data.get_all_values()
    sheet_fb = sheet.FB_data.get_all_values()
    sheet_googl = sheet.GOOGL_data.get_all_values()
    sheet_IBM = sheet.IBM_data.get_all_values()
    sheet_intc = sheet.INTC_data.get_all_values()
    sheet_msft = sheet.MSFT_data.get_all_values()
    sheet_orcl = sheet.ORCL_data.get_all_values()
    sheet_qcom = sheet.QCOM_data.get_all_values()
    sheet_tsla = sheet.TSLA_data.get_all_values()
    sheet_vz = sheet.VZ_data.get_all_values()

    sheet_aapl = sheet_aapl[1:] #remove column names title
    sheet_amzn = sheet_amzn[1:] #remove column names title
    sheet_csco = sheet_csco[1:] #remove column names title
    sheet_fb = sheet_fb[1:] #remove column names title
    sheet_googl = sheet_googl[1:] #remove column names title
    sheet_intc = sheet_intc[1:] #remove column names title
    sheet_msft = sheet_msft[1:] #remove column names title
    sheet_IBM = sheet_IBM[1:] #remove column names title
    sheet_orcl = sheet_orcl[1:] #remove column names title
    sheet_qcom = sheet_qcom[1:] #remove column names title
    sheet_tsla = sheet_tsla[1:] #remove column names title
    sheet_vz = sheet_vz[1:] #remove column names title


    
    AAPL_data = pd.DataFrame(sheet_aapl, columns = ['timestamp','symbol','tradingDay','open','high','low','close','volume']) 
    AMZN_data = pd.DataFrame(sheet_amzn, columns = ['timestamp','symbol','tradingDay','open','high','low','close','volume']) 
    CSCO_data = pd.DataFrame(sheet_csco, columns = ['timestamp','symbol','tradingDay','open','high','low','close','volume']) 
    FB_data = pd.DataFrame(sheet_fb, columns = ['timestamp','symbol','tradingDay','open','high','low','close','volume']) 
    GOOGL_data = pd.DataFrame(sheet_googl, columns = ['timestamp','symbol','tradingDay','open','high','low','close','volume']) 
    INTC_data = pd.DataFrame(sheet_intc, columns = ['timestamp','symbol','tradingDay','open','high','low','close','volume']) 
    MSFT_data = pd.DataFrame(sheet_msft, columns = ['timestamp','symbol','tradingDay','open','high','low','close','volume']) 
    IBM_data = pd.DataFrame(sheet_IBM, columns = ['timestamp','symbol','tradingDay','open','high','low','close','volume']) 
    ORCL_data = pd.DataFrame(sheet_orcl, columns = ['timestamp','symbol','tradingDay','open','high','low','close','volume']) 
    QCOM_data = pd.DataFrame(sheet_qcom, columns = ['timestamp','symbol','tradingDay','open','high','low','close','volume']) 
    TSLA_data = pd.DataFrame(sheet_tsla, columns = ['timestamp','symbol','tradingDay','open','high','low','close','volume']) 
    VZ_data = pd.DataFrame(sheet_vz, columns = ['timestamp','symbol','tradingDay','open','high','low','close','volume']) 


    return [AAPL_data, AMZN_data, CSCO_data, FB_data, GOOGL_data, INTC_data, MSFT_data, IBM_data, ORCL_data, QCOM_data, TSLA_data, VZ_data]

    
all_stock_data = getAllSheets()  ##THIS VAR HOLDS ALL OF THE STOCK DATA


# Reformat Server Data to Fit A.I. Model:

In [ ]:
df_AAPL = all_stock_data[0] #AAPL
df_AMZN = all_stock_data[1] #AMZN
df_CSCO = all_stock_data[2] #CSCO
df_FB = all_stock_data[3] #FB
df_GOOGL = all_stock_data[4] #GOOGL
df_INTC = all_stock_data[5] #INTC
df_MSFT = all_stock_data[6] #MSFT
df_IBM = all_stock_data[7] #IBM
df_ORCL = all_stock_data[8] #ORCL
df_QCOM = all_stock_data[9] #QCOM
df_TLSA = all_stock_data[10] #TSLA
df_VZ = all_stock_data[11] #VZ

all_stock_dataframes = [df_AAPL, df_AMZN, df_CSCO, df_FB, df_GOOGL, df_INTC, df_MSFT, df_IBM, df_ORCL, df_QCOM, df_TLSA, df_VZ]

In [ ]:
count = 0

for df in all_stock_dataframes:    
    df['open'] = df['open'].astype(float)
    df['high'] = df['high'].astype(float)
    df['low'] = df['low'].astype(float)
    df['close'] = df['close'].astype(float)
    df['volume'] = df['volume'].astype(int)
    for index, row in df.iterrows():    
        if row['timestamp'].endswith('-05:00'):
            df['timestamp'].iloc[index] = row['timestamp'][:-6]
               
    df.index = df['timestamp']
    print(count, "of ", len(all_stock_dataframes), "stock dataframes successfully reformatted")
    count+=1

print("All data successfully reformatted!")

In [ ]:
total_rows = df_AAPL.count()
print(total_rows['close'] +1)

training_data_split = int(total_rows['close']*0.6) #80% of the data becomes train data

In [ ]:
def determine_ticker(ticker):
    if (ticker == 'AAPL'):
        return df_AAPL
    if (ticker == 'AMZN'):
        return df_AMZN
    if (ticker == 'CSCO'):
        return df_CSCO
    if (ticker == 'FB'):
        return df_FB
    if (ticker == 'GOOGL'):
        return df_GOOGL
    if (ticker == 'INTC'):
        return df_INTC
    if (ticker == 'MSFT'):
        return df_MSFT
    if (ticker == 'IBM'):
        return df_IBM
    if (ticker == 'ORCL'):
        return df_ORCL
    if (ticker == 'QCOM'):
        return df_QCOM
    if (ticker == 'TLSA'):
        return df_TLSA
    if (ticker == 'VZ'):
        return df_VZ

# LONG SHORT TERM MEMORY MACHINE LEARNING (MULIPLE INPUTS):

In [36]:
def LSTM_prediction_multi_precision3(symbol, sensitivity, prediction_date , data_points):
    totalInputs = len(data_points)
    print("stock symbol:",symbol)
    print("sensitivity:",sensitivity)
    print("Date to be predicted: ", prediction_date)
    x = datetime.datetime(prediction_date[0:4], 3, 23)
    print(x)
    print(x.weekday())

    print("total Inputs:",totalInputs)
    
    
    current_stock_df = determine_ticker(symbol)
    
    #creating dataframe
    data = current_stock_df.sort_index(ascending=True, axis=0)
    new_data = pd.DataFrame(index=range(0,len(current_stock_df)),columns=['timestamp', data_points[0], data_points[1],data_points[2],data_points[3],data_points[4]])

    
    #creating a new_data DF for manipulation
    for i in range(0,len(data)):
        new_data['timestamp'][i] = data['timestamp'][i]
        new_data[data_points[0]][i] = data[data_points[0]][i]
        new_data[data_points[1]][i] = data[data_points[1]][i]
        new_data[data_points[2]][i] = data[data_points[2]][i]
        new_data[data_points[3]][i] = data[data_points[3]][i]
        new_data[data_points[4]][i] = data[data_points[4]][i]

        
    #setting index to timestamp 
    new_data.index = new_data.timestamp
    new_data.drop('timestamp', axis=1, inplace=True)


    #creating train and test sets
    dataset = new_data.values 
    
    
    train = dataset[0:training_data_split,:] #TRAINGING 80%
    valid = dataset[training_data_split:,:] #DATA POINTS TO BE PREDICTED 20%
    
    #converting dataset into x_train and y_train
    scaler = MinMaxScaler(feature_range=(0, 1))   #NORMALIZARION between O and 1 of the data
    scaled_data = scaler.fit_transform(dataset)   #FITS OUR DATA SET TO THE NORMALIZATION BETWEEN ONE AND ZERO
      
    #Spliting the data into X train and Y Train
    x_train, y_train = [], []
    for i in range(50,len(train)):
        x_train.append(scaled_data[i-50:i,:])
        y_train.append(scaled_data[i,1])

    
    x_train, y_train = np.array(x_train), np.array(y_train)


    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],totalInputs))  
  
    print("x_train.shape:", x_train.shape)
  #  print("x_train.shape:", x_train)
    
    # create and fit the LSTM network
    model = Sequential() 
    print(type(model))

    model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1],totalInputs))) #2 input
    model.add(LSTM(units=30, return_sequences=True))
    model.add(LSTM(units=30))
    model.add(Dense(units=1))
    model.summary()
    
    print(x_train.shape)

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(x_train, y_train, epochs=sensitivity, batch_size=32)  #EPOCHS determine sensitivity
    model.summary()
    
    #predicting values, using past 50 from the train data
    inputs = new_data[len(new_data) - len(valid) - 50:].values
    
    inputs = inputs.reshape(-1,totalInputs)   
    inputs  = scaler.transform(inputs)

    X_test = []
    for i in range(50,inputs.shape[0]):
        X_test.append(inputs[i-50:i,:])
   

    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],totalInputs))

    predictons = model.predict(X_test) #PREDICTIONS

    formated_preds = np.zeros((predictons.shape[0],predictons.shape[1]+(totalInputs-1)))
    formated_preds[:,:-1] = predictons

    predictons = scaler.inverse_transform(formated_preds) #DE NORMALIZARION
   


    #for plotting
    train = new_data[:training_data_split]
    valid = new_data[training_data_split:]
   
    valid['Predictions'] = predictons[: , :1]
    plt.figure(figsize=(15,10))


    plt.plot(train[data_points[0]], 'b')
    plt.plot(valid[[data_points[0],'Predictions']])

    plt.xlabel('timestamp')
    plt.ylabel(data_points[0])
    plt.title(symbol) 

In [37]:
LSTM_prediction_multi_precision3('AAPL', 10, 20190304, ['open','close','low','high','volume'])
#LSTM_prediction_multi_precision3('AMZN', 30, ['open','close','low','high','volume'])
#LSTM_prediction_multi_precision3('CSCO', 30, ['open','close','low','high','volume'])
#LSTM_prediction_multi_precision3('FB', 30, ['open','close','low','high','volume'])
#LSTM_prediction_multi_precision3('GOOGL', 30, ['open','close','low','high','volume'])
#LSTM_prediction_multi_precision3('INTC', 30, ['open','close','low','high','volume'])
#LSTM_prediction_multi_precision3('MSFT', 30, ['open','close','low','high','volume'])
#LSTM_prediction_multi_precision3('IBM', 30, ['open','close','low','high','volume'])
#LSTM_prediction_multi_precision3('ORCL', 30, ['open','close','low','high','volume'])
#LSTM_prediction_multi_precision3('QCOM', 30, ['open','close','low','high','volume'])
#LSTM_prediction_multi_precision3('TLSA', 30, ['open','close','low','high','volume'])
#LSTM_prediction_multi_precision3('VZ', 30, ['open','close','low','high','volume'])

#Stock OPTIONS: AAPL, AMZN, CSCO, FB, GOOGL,INTC, MSFT, IBM, ORCL, QCOM, TLSA, VZ
#predictionDate in form YYYYMMDD - 20190304

stock symbol: AAPL
sensitivity: 10
Date to be predicted:  20190304


TypeError: 'int' object is not subscriptable

# SPLIT BY DAY

In [ ]:

new_df = df
daysList = [];

def createDayDF():
    global new_df
    day_df = pd.DataFrame(columns = ['timestamp','symbol','tradingDay','open','high','low','close','volume'])
    currentDay = new_df.iloc[0]['tradingDay']
    #print(currentDay)
    
    for index, row in new_df.iterrows():    
        if row['tradingDay'] == currentDay:
            day_df = day_df.append(row)
            #print(index)
            new_df = new_df.drop([index])

    daysList.append(day_df)
    
while(not new_df.empty): #splitting up data by data and sorting it in an array 
    createDayDF()

print(len(daysList))


def getLastDayInTable():
    print(daysList[-1].iloc[-1]['tradingDay'])
    return daysList[-1]
getLastDayInTable()


In [ ]:
fig = plt.figure(figsize=(30,10))


for i in range(len(daysList)): 
    #print(daysList[i]) 
    plt.subplot(2, 9, i+1)
    #plt.plot(x, y)
    
    plt.plot(daysList[i]['low'], 'b')
        
    plt.xlabel('TIMESTAMP')
    plt.ylabel('PRICE')
    plt.title('AAPL PRCIES ' + str(daysList[i]['tradingDay'].iloc[0])) 


plt.show()

In [ ]:

copy_df = df
daysList = [];
weeksList = [];

def createDayDF():
    global copy_df
    day_df = pd.DataFrame(columns = ['timestamp','symbol','tradingDay','open','high','low','close','volume'])

    currentDay = copy_df.iloc[0]['tradingDay']

    
    for index, row in copy_df.iterrows():    
        if row['tradingDay'] == currentDay:
            day_df = day_df.append(row)
            #print(index)
            copy_df = copy_df.drop([index])

    daysList.append(day_df)
    
    

def createWeekDF():
    week_df = pd.DataFrame(columns = ['timestamp','symbol','tradingDay','open','high','low','close','volume'])
    print()
    newWeekLength = 0 #used for days that the market is closed
    for i in range(len(daysList)): 
        currentDay = daysList[i].iloc[0]['tradingDay']
        currentDay_raw = currentDay.replace('-','')
       # print((currentDay_raw))
        dt = datetime.datetime(int(currentDay_raw[0:4]), int(currentDay_raw[4:6]), int(currentDay_raw[6:8]))
       # print(dt)
        #print(dt.weekday())
        
        
        if (len(weeksList) == 0):
            print("First week created - starting with " , dt.weekday())
            weeksList.append(daysList[i])
            
        elif (newWeekLength >= 5):
            print("more then 5, new week - starting with " , dt.weekday())
            weeksList.append(daysList[i])
            newWeekLength = 0
        else:
            if (dt.weekday() == 0): #monday new week
                print("--------")
                print("Creating New week - day ", dt.weekday())
                weeksList.append(daysList[i])
                newWeekLength = 0
            else:
                print("adding to week", dt.weekday())
                weeksList[-1] = pd.concat([weeksList[-1],daysList[i]])
                 
            #0 = monday
            #1 = tues
            #2 = wed
            #3 = thurs
            #4 = fri
        newWeekLength += 1
        
while (not copy_df.empty): #splitting up data by data and sorting it in an array 
    createDayDF()

    
createWeekDF()
#print(weeksList[0])

In [ ]:
fig = plt.figure(figsize=(15,30))


for i in range(len(weeksList)): 
    #print(daysList[i]) 
    plt.subplot(5, 1, i+1)
    #plt.plot(x, y)
    
    plt.plot(weeksList[i]['close'], 'b')

    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=5)
    
    plt.xlabel('TIMESTAMP')
    plt.ylabel('PRICE')
    plt.title('AAPL PRICES FROM: ' + str(weeksList[i]['tradingDay'].iloc[0]) + " TO " + str(weeksList[i]['tradingDay'].iloc[-1])) 
    

plt.show()